In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mlxtend
from mlxtend.frequent_patterns import association_rules, apriori

In [2]:
df = pd.read_csv("Bakery-sales.csv", low_memory=False)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   idx            234005 non-null  int64  
 1   date           234005 non-null  object 
 2   time           234005 non-null  object 
 3   ticket_number  234005 non-null  float64
 4   article        234005 non-null  object 
 5   Quantity       234005 non-null  object 
 6   unit_price     234005 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 12.5+ MB
None


,idx,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,0.90
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,1.20
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,1.20
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,1.15
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,1.20


In [12]:
# Encoding basket
df_basket = df.groupby('ticket_number')['article'].apply(set).to_frame()

selected_item = ["TRADITIONAL BAGUETTE", "CROISSANT", "PAIN AU CHOCOLAT", 
                 "COUPE", "BANETTE", "CEREAL BAGUETTE", "SPECIAL BREAD", 
                 "BOULE 400G", "FORMULE SANDWICH", "TARTELETTE", "CAMPAGNE", 
                 "VIK BREAD", "COOKIE", "BOULE 200G", "FICELLE", "COMPLET", "KOUIGN AMANN", "MOISSON", "BANETTINE"]

def filter_transaction(x):
    for i in x:
        if i in selected_item:
            return 1
    return 0

df_basket['filter'] = df_basket['article'].apply(filter_transaction)
df_basket = df_basket[df_basket['filter'] == 1].drop('filter', axis=1)

basket_encoded = df_basket['article'].str.join('|').str.get_dummies('|')


# Create frequent itemset
itemsets = apriori(basket_encoded, min_support=0.01, use_colnames=True)

# Convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.2)

rules.sort_values(by=['lift'], ascending=False)\
    .drop(columns=['antecedent support', 'consequent support', 'conviction'])\
    .head(6)

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift,leverage,zhangs_metric
20,(VIK BREAD),"(TRADITIONAL BAGUETTE, COUPE)",0.010134,0.373960,7.053579,0.008697,0.882132
22,"(TRADITIONAL BAGUETTE, PAIN AU CHOCOLAT)",(CROISSANT),0.019660,0.539358,5.435942,0.016044,0.846910
3,(BOULE 200G),(COUPE),0.020675,0.888268,5.275418,0.016756,0.829754
12,(CROISSANT),(PAIN AU CHOCOLAT),0.046758,0.471256,5.163231,0.037702,0.895139
11,(PAIN AU CHOCOLAT),(CROISSANT),0.046758,0.512299,5.163231,0.037702,0.887309
21,"(TRADITIONAL BAGUETTE, CROISSANT)",(PAIN AU CHOCOLAT),0.019660,0.460321,5.043417,0.015762,0.837491


In [52]:
# Encoding basket
df_basket = df.groupby('ticket_number')['article'].apply(set).to_frame()

basket_encoded = df_basket['article'].str.join('|').str.get_dummies('|')

selected_item = ["TRADITIONAL BAGUETTE", "CROISSANT", "PAIN AU CHOCOLAT", 
                 "COUPE", "BANETTE", "CEREAL BAGUETTE", "SPECIAL BREAD", 
                 "BOULE 400G", "FORMULE SANDWICH", "TARTELETTE", "CAMPAGNE", 
                 "VIK BREAD", "COOKIE", "BOULE 200G", "FICELLE", "COMPLET", "KOUIGN AMANN", "MOISSON", "BANETTINE"]

# Create frequent itemset
itemsets = apriori(basket_encoded, min_support=0.01, use_colnames=True)

# Convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.2)

rules.sort_values(by=['lift'], ascending=False)\
    .drop(columns=['antecedent support', 'consequent support', 'conviction'])\
    .head(6)

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift,leverage,zhangs_metric
13,"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",(CROISSANT),0.016621,0.539358,6.429838,0.014036,0.871327
0,(BOULE 200G),(COUPE),0.017479,0.888268,6.239965,0.014678,0.856598
9,(CROISSANT),(PAIN AU CHOCOLAT),0.039531,0.471256,6.107265,0.033058,0.912832
8,(PAIN AU CHOCOLAT),(CROISSANT),0.039531,0.512299,6.107265,0.033058,0.906185
15,"(TRADITIONAL BAGUETTE, CROISSANT)",(PAIN AU CHOCOLAT),0.016621,0.460321,5.965544,0.013835,0.863552
16,(PAIN AU CHOCOLAT),"(TRADITIONAL BAGUETTE, CROISSANT)",0.016621,0.215405,5.965544,0.013835,0.901970


In [17]:
rules[rules['lift'] > 1].reset_index(drop=True).sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(BOULE 200G),(COUPE),0.023275,0.168379,0.020675,0.888268,5.275418,0.016756,7.443011,0.829754
4,(CAMPAGNE),(COUPE),0.033738,0.168379,0.026977,0.799589,4.748753,0.021296,4.149577,0.816982
3,(BOULE 400G),(COUPE),0.035385,0.168379,0.028104,0.794219,4.716858,0.022145,4.041283,0.816901
5,(COMPLET),(COUPE),0.027159,0.168379,0.020059,0.738589,4.386476,0.015486,3.181281,0.793579
8,(VIK BREAD),(COUPE),0.027098,0.168379,0.019886,0.733845,4.358301,0.015323,3.124577,0.792015
11,"(VIK BREAD, TRADITIONAL BAGUETTE)",(COUPE),0.013861,0.168379,0.010134,0.731082,4.341890,0.007800,3.092471,0.780504
6,(MOISSON),(COUPE),0.026829,0.168379,0.019140,0.713409,4.236929,0.014623,2.901768,0.785042
7,(SPECIAL BREAD),(COUPE),0.044921,0.168379,0.026431,0.588383,3.494402,0.018867,2.020376,0.747402
14,"(TRADITIONAL BAGUETTE, PAIN AU CHOCOLAT)",(CROISSANT),0.036451,0.099221,0.019660,0.539358,5.435942,0.016044,1.955486,0.846910
9,(PAIN AU CHOCOLAT),(CROISSANT),0.091272,0.099221,0.046758,0.512299,5.163231,0.037702,1.846992,0.887309


In [18]:
rules.to_csv("result.csv", index=False)